# Constraint satisfaction problem

In the previous tasks the state was opaque: an algorithm could execute an action and transform a state to another state, but that was it. It could not depend in its search on the details of a state, because the state representation was different from problem to problem. In CSP this is no longer true: the state always consists of an assignment of values over some domains to variables. This enables an algorithm to interact with the state directly, not only through opaque actions. 

Allowed assignments are limited by constraints in the problem and broadly speaking they can be divided into three groups:

* *Unary constraints* each concerning only a single variable (e.g., $x>0$)
* *Binary constraints* relating two variables to each other (e.g., $x>y$)
* *General constraints* concerning more than two variables at once

In general, the shape of constraints is unrestricted and we are not limited to numeric domains and constraints in the form of inequalities. In principle, one could employ arbitrary boolean functions as constraints.

The following class `CSP` is a very limited representation of such a problem capable only of handling:

* finite domains that can be materialized (e.g., the set of all 32-bit integers is doable in theory; the set of all 64-bit integers is out of reach) 
* unary constraints by setting the domain of a variable appropriately;
* binary constraints defined in terms of Python's operators `==`, `!=`, `<`, `<=`, `>` and `>=` such that both sides are variables.

About its methods:

* The constructor (`__init__`) requires no arguments.
* `addVariable` requires two arguments: 
    * `var` - a variable (a string representing its name, but really, it could be any object);
    * `domain` - a collection of the allowed values for the variable
* `addBinaryConstraint` also requires three arguments:
    * `var1` - a variable, the left-hand side of the constraint
    * `op` - an operator (one of the following strings: `'=='`, `'!='`, `'<'`, `'<='`, `'>'`, `'>='`)
    * `var2` - a variable, the right-hand side of the constraint
* `verify` is used to verify whether the relation given by the second argument `op` holds between the values given as the first argument `left` and the third argument `right`. Returns `True` if it does and `False` otherwise.
* `is_complete` gets a single argument: a dictionary representing an assignment of values to variables. From now on, we call such a dictionary an *assignment*. It returns `True` if all the variables of the problem are present in the dictionary, and `False` otherwise.
* `is_consistent` also requries an assignment. It returns `True` if all the values in the assignment are such that they come from the respecitve domains, and no binary constrain is violated (but if the assignment is not complete then some may be unfulfilled).

In [3]:
class CSP:
    def __init__(self):
        self.domains = {}
        self.binary = []
    
    def addVariable(self, var, domain):        
        assert var not in self.domains
        self.domains[var] = set(domain)
        
    def addBinaryConstraint(self, var1, operator, var2):
        assert var1 in self.domains
        assert var2 in self.domains
        c = (var1, operator, var2)
        self.binary.append(c)      
        
    def verify(self, left, op, right):
        if op[0] == '=':
            return left == right
        if op == '!=':
            return left != right
        if op == '<':
            return left < right
        if op == '<=':
            return left <= right
        if op == '>':
            return left > right
        if op == '>=':
            return left >= right
        
    def is_complete(self, assignment):
        return self.domains.keys() <= assignment.keys() 
        
    def is_consistent(self, assignment):
        for var, value in assignment.items():            
            if value not in self.domains[var]:
                return False
        for var1, op, var2 in self.binary:
            if var1 in assignment and var2 in assignment:
                if not self.verify(assignment[var1], op, assignment[var2]):
                    return False
        return True

## Making Australia colorful again

Lets model a map coloring problem. In particular, we'll color the following map of Australia. It can be simplified to a graph presented in the right-hand side of the picture below.

![](aima-images/fig_6_1.png)

We begin by creating a new problem in the variable `australia` over the domain consisting of three values: `'R'` statnding for the color red, `'G'` standing for green and `'B'` standing for blue. Of course the particular symbols used are not relevant. There are 7 teritories, so we add 7 variables to the problem.

Any two teritories of Australia sharing a border should be colored using different colours. We thus define 9 binary constraints.

In [4]:
australia = CSP()
australia.addVariable('WA', {'R', 'G', 'B'})
australia.addVariable('NT', {'R', 'G', 'B'})
australia.addVariable('SA', {'R', 'G', 'B'})
australia.addVariable('Q', {'R', 'G', 'B'})
australia.addVariable('NSW', {'R', 'G', 'B'})
australia.addVariable('V', {'R', 'G', 'B'})
australia.addVariable('T', {'R', 'G', 'B'})

In [5]:
australia.addBinaryConstraint('WA', '!=', 'NT')
australia.addBinaryConstraint('WA', '!=', 'SA')
australia.addBinaryConstraint('NT', '!=', 'SA')
australia.addBinaryConstraint('NT', '!=', 'Q')
australia.addBinaryConstraint('SA', '!=', 'Q')
australia.addBinaryConstraint('SA', '!=', 'NSW')
australia.addBinaryConstraint('SA', '!=', 'V')
australia.addBinaryConstraint('Q', '!=', 'NSW')
australia.addBinaryConstraint('NSW', '!=', 'V')

Let's see how the `verify` method works.

In [6]:
print("Is G=G?", australia.verify('G', '==', 'G'))
print("Is 5<7?", australia.verify(5, '<', 7))
print("Is 5>=7?", australia.verify(5, '>=', 7))

Is G=G? True
Is 5<7? True
Is 5>=7? False


Lets now consider an assignment such that *Western Australia* is red and the rest is unspecified. Is it a consistent assignment?

In [7]:
australia.is_consistent({'WA': 'R'})

True

Is it a complete assignment?

In [8]:
australia.is_complete({'WA': 'R'})

False

As expected it didn't work: there are still 9 unfulfiled constraints. So lets extend the solution.

In [9]:
assignment = {'WA': 'R', 'Q': 'R', 'V': 'R', 'NT': 'B', 'NSW': 'B', 'SA': 'G'}
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))

Is consistent? True
Is complete? False


Now it almost works: all the constraints are satisfied, but the solution is still not complete, as Tasmania is still colorless.

In [10]:
assignment = {'WA': 'R', 'Q': 'R', 'V': 'R', 'NT': 'B', 'NSW': 'B', 'SA': 'G', 'T': 'B'}
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))

Is consistent? True
Is complete? True


Finally, we have manually collored the map of Australia! Now let's make computer do this for us.

## Task 1: Backtracking

Complete the following cell of code by implementing the recursive backtracking algorithm. Basically, it is a recursive DFS where an action is assigning a value to a variable. Avoid making copies of assignments, instead operate on a single assignment, and update it accordingly. Increment `self.counter` every time you check wheter an assigment is a solution (i.e., is consistent and complete). Use the provided methods `select_unassigned_variable` and `order_domain_values` to, respectively, get a next variable to assign a value to, and to get the list of allowed values for the variable. `solve` should return a solution (i.e., a complete, consistent assignment) or `None`.

In [11]:
from copy import deepcopy

class RecursiveBacktracking:
    def __init__(self, csp):
        self.csp = csp
        self.counter = 0
        
    def select_unassigned_variable(self, assignment):
        for var in self.csp.domains.keys():
            if var not in assignment:
                return var
        return None
    
    def order_domain_values(self, variable, assignment):
        return self.csp.domains[variable]

    # (⬇️) Simple backtracking method    
    def backtracking (self, assignment):

        # self.counter += 1
        if self.csp.is_complete(assignment):
            return assignment
        
        variable = self.select_unassigned_variable(assignment)
        for value in self.order_domain_values(variable, assignment):
            assignment[variable] = value
            self.counter += 1
            if self.csp.is_consistent(assignment):
                sol = self.backtracking(assignment)
                if sol:
                    return sol
            del assignment[variable]


    def solve(self):
        self.counter = 0
        return self.backtracking({})

        

Let's test it on the Australia problem! In the cell below you should get a consistent, complete assignment, i.e., a solution to the problem.

In [12]:
solver = RecursiveBacktracking(australia)
assignment = solver.solve()
print("Assignment", assignment)
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))
print("# considered assignments", solver.counter)

Assignment {'WA': 'R', 'NT': 'B', 'SA': 'G', 'Q': 'R', 'NSW': 'B', 'V': 'R', 'T': 'R'}
Is consistent? True
Is complete? True
# considered assignments 11


## Task 2: Extend `RecursiveBacktracking` with the MRV and degree heuristics

Modify your `RecursiveBacktracking` to include the MRV (minimum remaining values) heuristics and the degree heuristics. Implement them in the appropriate places below in the class `RecursiveBacktrackingWithHeuristics`. If you want, you can also implement the least constraining value heuristics, but this is not mandatory. Follow the same assumptions as for `RecursiveBacktracking`, so expose the main backtracking algorithm as the `solve` and keep track of the number of considered assignments in the `counter` variable.

In [13]:
from copy import deepcopy
from queue import PriorityQueue

class RecursiveBacktrackingWithHeuristics:
    def __init__(self, csp):
        self.csp = csp
        self.counter = 0
    
    #(⬇️) method to reduce copy of domains
    def propagate(self, domains, constrains, variable, value):
        new_domains = deepcopy(domains)
        new_constrains = constrains.copy()
        for var1, op, var2 in constrains:
            if var1 == variable:
                for v in domains[var2]:
                    if not self.csp.verify(value,op, v):
                        new_domains[var2].discard(v)
                        new_constrains.remove((var1,op,var2))

            if var2 == variable:
                for v in domains[var1]:
                    if not self.csp.verify(v,op,value):
                        new_domains[var1].discard(v)
                        new_constrains.remove((var1,op,var2))
        del new_domains[variable]
        new_domains[variable] = value
        return new_domains, new_constrains

    #(⬇️) selecting variable that has least ammount of avaliable values in copy of domains (if equal choose one with most constrains on other variables)
    def select_unassigned_variable(self, assignment, domains, constrains):
        maxi = None
        degree = 0
        variable = None
        for var in domains:
            if var not in assignment:
                if not maxi:
                    degree = 0
                    variable = var
                    maxi = len(domains[var])
                else:
                    if maxi > len(domains[var]):
                        degree = 0
                        variable = var
                        maxi = len(domains[var])
                        for constrain in constrains:
                            if var in constrain:
                                degree += 1
                    if maxi == len(domains[var]):
                        temp_degree = 0
                        for var1, op, var2 in constrains:
                            if var == var1 and var2 not in assignment:
                                for v1 in domains[var1]:
                                    for v2 in domains[var2]:
                                        if not self.csp.verify(v1, op, v2):
                                            temp_degree += 1
                            if var == var2 and var1 not in assignment:
                                for v1 in domains[var2]:
                                    for v2 in domains[var1]:
                                        if not self.csp.verify(v1, op, v2):
                                            temp_degree += 1

                        if temp_degree > degree:
                            variable = var
                            degree = temp_degree

                    if len(domains[var]) == 0:
                        return None
        return variable
    
    #(⬇️) return values of an variable in least constraining order
    def order_domain_values(self, variable, domains, constrains):
        order_queue = PriorityQueue()
        for value in domains[variable]:
            temp = 0
            for var1, op, var2 in constrains:
                if var1 == variable:
                    for value2 in domains[var2]:
                        if self.csp.verify(value, op, value2):
                            temp+=1
                if var2 == variable:
                    for value2 in domains[var1]:
                        if self.csp.verify(value2, op, value):
                            temp+=1
            order_queue.put((temp, value))
        return list(x[1] for x in order_queue.queue)

    #(⬇️) backtracking fucntion combining heuristics selection, order and propagate in order to make it run faster and omit checking if assignment is consistent,  
    # instead it returns order of variable as none when chosen variable has no avaliable values     
    def backtracking (self, assignment, domains, constrains):
        if self.csp.is_complete(assignment):
            return assignment
        self.counter += 1
        variable = self.select_unassigned_variable(assignment, domains, constrains)
        if variable != None:
            order = self.order_domain_values(variable, domains, constrains)
            if order:
                for value in order:
                    assignment[variable] = value
                    new_domains, new_constrains = self.propagate(domains, constrains, variable, value)
                    sol = self.backtracking(assignment,new_domains, new_constrains)
                    if sol:
                        return sol
                    del assignment[variable]


    def solve(self):
        self.counter = 0
        domains = deepcopy(self.csp.domains)
        constrains = self.csp.binary.copy()
        return self.backtracking({}, domains, constrains)

Lets test your implementation by solving the problem of coloring Australia!

In [14]:
solver = RecursiveBacktrackingWithHeuristics(australia)
assignment = solver.solve()
print("Assignment", assignment)
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))
print("# considered assignments", solver.counter)

Assignment {'SA': 'B', 'NT': 'G', 'Q': 'R', 'NSW': 'G', 'WA': 'R', 'V': 'R', 'T': 'B'}
Is consistent? True
Is complete? True
# considered assignments 7


**Compare the number of considered assignments for `RecursiveBacktracking` and `RecursiveBacktrackingWithHeuristics`. How do they differ? Why do you think is that?**

*Your answer goes here*

## Sudoku!

Coloring Australia is really a bit boring. Lets deal with something more exciting: SUDOKU! I assume you know the rules, but if you don't, go look them up, spend some time solving a few puzzles and familiarize yourself with the puzzle.
The cell below defines a string `puzzle` which represents a Sudoku puzzle from the picture below. A digit in `puzzle` corresponds to a digit in the puzzle, while an underscore `_` corresponds to a blank cell in the puzzle. For your convenience there's also variable `solution` containing the correct solution to this particular puzzle.

![](aima-images/fig_6_4.png)

In [15]:
#(⬇️) easiest sudoku no need to backtrack
puzzle = '''
__3_2_6__
9__3_5__1
__18_64__
__81_29__
7_______8
__67_82__
__26_95__
8__2_3__9
__5_1_3__
'''

#(⬇️) a little harder sudoku where algorithm needs to backtrack
# puzzle = '''
# 9_____7__
# _4__3____
# __8__2_91
# 8__9_____
# _______6_
# __9__1_25
# 3___2__57
# __5___6__
# _____78__
# '''

#(⬇️) hardest sudoku, neeeds a lot of steps to solve
# puzzle = '''
# 8________
# __36_____
# _7__9_2__
# _5___7___
# ____457__
# ___1___3_
# __1____68
# __85___1_
# _9____4__
# '''

In [16]:
solution = '''
483921657
967345821
251876493
548132976
729564138
136798245
372689514
814253769
695417382
'''

## Task 3: Sudoku as CSP

Using the string `puzzle` defined above write in the cell below code that will add all the necessary constraints to the CSP represented by the variable `sudoku`. If you can and want, write a parser that will construct the CSP from a string representation of the puzzle. Otherwise, you can construct the problem manually.

In [17]:
sudoku = CSP()
tab=[]
for i in range(9):
    temp = []
    for j in range(9):
        temp.append(0)
    tab.append(temp)

for i in range(9):
    for j in range(9):
        if puzzle[i*10+j+1] == '_':
            sudoku.addVariable((i,j), {'1','2','3','4','5','6','7','8','9'})
        else:
            sudoku.addVariable((i,j), {puzzle[i*10+j+1]})

for i in range(9):
    for j in range(9):
        temp = i+1
        while temp < 9:
            sudoku.addBinaryConstraint((i,j), '!=' , (temp,j))
            temp += 1

        temp = j+1
        while temp<9:
            sudoku.addBinaryConstraint((i,j), '!=' , (i,temp))
            temp += 1
        square_n = (j//3)*3
        square_m = (i//3)*3
        for m in range(3):
            for n in range(3):
                if ((i,j), '!=', (m + square_m,n+square_n )) not in sudoku.binary and ((m + square_m, n+square_n), '!=',(i,j) ) not in sudoku.binary and (square_n+n != j or square_m+m != i):
                    sudoku.addBinaryConstraint((i,j), '!=', (m+square_m,n+square_n))

# mess

When I wrote this parser, only God and I knew how it worked. Now, only God knows it!
I think it worked fine when I wrote it so if it breaks I have no idea why. 

In [23]:
#(⬇️) poor atempt at visualising sudoku :(
def assignment_visualisation (assignment):
    for i in range(9):
        for j in range(9):
            print(assignment[(i,j)], end=' ')
            if j%3 == 2 and j != 8:
                print('|',end=' ')
        if i%3 == 2 and i != 8:
            print()
            print('------+-------+------',end='')
        print()
    print()

Let's see if it works... Unless you did some clever tricks and/or are quite patient - it does not, it is too slow.

In [24]:
solver = RecursiveBacktrackingWithHeuristics(sudoku)
assignment = solver.solve()
assignment_visualisation(assignment)
print("Assignment", assignment)
print("Is consistent?", sudoku.is_consistent(assignment))
print("Is complete?", sudoku.is_complete(assignment))
print("# considered assignments", solver.counter)

4 8 3 | 9 2 1 | 6 5 7 
9 6 7 | 3 4 5 | 8 2 1 
2 5 1 | 8 7 6 | 4 9 3 
------+-------+------
5 4 8 | 1 3 2 | 9 7 6 
7 2 9 | 5 6 4 | 1 3 8 
1 3 6 | 7 9 8 | 2 4 5 
------+-------+------
3 7 2 | 6 8 9 | 5 1 4 
8 1 4 | 2 5 3 | 7 6 9 
6 9 5 | 4 1 7 | 3 8 2 

Assignment {(0, 2): '3', (0, 4): '2', (0, 6): '6', (1, 0): '9', (1, 3): '3', (1, 5): '5', (1, 8): '1', (2, 2): '1', (2, 3): '8', (2, 5): '6', (2, 6): '4', (3, 2): '8', (3, 3): '1', (3, 5): '2', (3, 6): '9', (4, 0): '7', (4, 8): '8', (5, 2): '6', (5, 3): '7', (5, 5): '8', (5, 6): '2', (6, 2): '2', (6, 3): '6', (6, 5): '9', (6, 6): '5', (7, 0): '8', (7, 3): '2', (7, 5): '3', (4, 5): '4', (7, 8): '9', (8, 2): '5', (4, 2): '9', (4, 3): '5', (8, 3): '4', (0, 3): '9', (2, 4): '7', (0, 5): '1', (1, 4): '4', (1, 2): '7', (1, 6): '8', (1, 7): '2', (1, 1): '6', (7, 2): '4', (8, 4): '1', (6, 4): '8', (7, 4): '5', (8, 5): '7', (8, 6): '3', (4, 6): '1', (7, 6): '7', (6, 8): '4', (6, 7): '1', (6, 0): '3', (6, 1): '7', (7, 1): '1', (7, 7): '6', (4, 7): 

## Task 4: Solving sudoku

In order to make it working, we need to extend backtracking with inference. Complete the following class `RecursiveBacktrackingWithAC3` in such a way that it uses the same heuristics as `RecursiveBacktrackingWithHeuristics` and on top of that uses the AC3 algorithm after a new value is added to the assignment.

In this task, you don't need to stick to the previous API with method. A constructor with a single argument, and an argumentless `solve` are sufficient, heuristics and AC3 can be handled however you want.

In [25]:
from queue import Queue

class RecursiveBacktrackingWithAC3:
    def __init__(self, csp):
        self.csp = csp
        self.counter = 0

    #(⬇️) same logic as in backtracking in heuristics
    def select_unassigned_variable(self, assignment, domains, constrains):
        maxi = None
        degree = 0
        variable = None
        for var in domains:
            if var not in assignment:
                if not maxi:
                    degree = 0
                    variable = var
                    maxi = len(domains[var])
                else:
                    if maxi > len(domains[var]):
                        degree = 0
                        variable = var
                        maxi = len(domains[var])
                        for constrain in constrains:
                            if var in constrain:
                                degree += 1
                    if maxi == len(domains[var]):
                        temp_degree = 0
                        for var1, op, var2 in constrains:
                            if var == var1 and var2 not in assignment:
                                for v1 in domains[var1]:
                                    for v2 in domains[var2]:
                                        if not self.csp.verify(v1, op, v2):
                                            temp_degree += 1
                            if var == var2 and var1 not in assignment:
                                for v1 in domains[var2]:
                                    for v2 in domains[var1]:
                                        if not self.csp.verify(v1, op, v2):
                                            temp_degree += 1

                        if temp_degree > degree:
                            variable = var
                            degree = temp_degree

                    if len(domains[var]) == 0:
                        return None
        return variable
    
    #(⬇️) same logic as in backtracking in heuristics
    def order_domain_values(self, variable, domains, constrains):
        order_queue = PriorityQueue()
        for value in domains[variable]:
            temp = 0
            for var1, op, var2 in constrains:
                if var1 == variable:
                    for value2 in domains[var2]:
                        if self.csp.verify(value, op, value2):
                            temp+=1
                if var2 == variable:
                    for value2 in domains[var1]:
                        if self.csp.verify(value2, op, value):
                            temp+=1
            order_queue.put((temp, value))
        return list(x[1] for x in order_queue.queue)
        
    #(⬇️) remove inconsistent values of a variable, returs true if someone was deleted, false if not
    def Remove_inconsistent_values(self, var1, var2, domains, v1, op, v2):
        removed = False
        toremove = []
        nonconsistent = True
        for value1 in domains[var1]:
            nonconsistent = True
            for value2 in domains[var2]: 
                    if v1 == var1 and v2 == var2:
                        if self.csp.verify(value1, op , value2):
                            nonconsistent = False
                    if v2 == var1 and v1 == var2:
                        if self.csp.verify(value2, self.reverse_operator(op), value1):
                            nonconsistent = False
            if nonconsistent:
                toremove.append((var1,value1))
                removed = True
        for var, value in toremove:
            if len(domains[var])>1:
                domains[var].remove(value)
            else:
                domains[var] = set()

        return removed

    #(⬇️) reverse sign 
    def reverse_operator(self, op):
        if op == '>':
            return '<'
        if op == '>=':
            return '<='
        if op == '<':
            return '>'
        if op == '<=':
            return '>='
        if op == '!=':
            return '!='
        if op[0] == '=':
            return '=='

    #(⬇) main ac3 logic
    def AC3(self, domains, assignment, constrains):
        arcs = Queue()
        for var1, op, var2 in constrains:
            if var1 not in assignment or var2 not in assignment:
                arcs.put((var1, op, var2))
                arcs.put((var2, self.reverse_operator(op), var1))
        while(not arcs.empty()):
            var1, op, var2 = arcs.get()
            if self.Remove_inconsistent_values(var1, var2, domains, var1, op, var2):
                for v1, op, v2 in constrains:
                    if v1 not in assignment and v2 not in assignment:
                        if v1 == var1:
                            arcs.put((v2, op, var1))
                        if v2 == var1:
                            arcs.put((v1, op, var1))
        return domains

    #(⬇️) backtracking implementing all of above methods to restrict domains more then just heuristics in order to find solution with less steps (if possible)
    def backtracking (self, assignment, domains, constrains):
        if self.csp.is_complete(assignment):
            return assignment
        
        variable = self.select_unassigned_variable(assignment, domains, constrains)
        if variable != None:
            order = self.order_domain_values(variable, domains, constrains)
            if order:
                for value in order:
                    assignment[variable] = value
                    self.counter += 1
                    # if self.csp.is_consistent(assignment):
                    new_domains = deepcopy(domains)
                    new_domains[variable] = value
                    new_domains = self.AC3(new_domains, assignment, constrains)
                    sol = self.backtracking(assignment,new_domains, constrains)
                    if sol:
                        return sol
                    del assignment[variable]
    
    def solve(self):
        self.counter = 0
        domains = deepcopy(self.csp.domains)
        constrains = self.csp.binary.copy()
        return self.backtracking({}, domains, constrains)
        

Let's test it on both considered problem. If everything went all right, both cells should terminate within few seconds at most, and you should see that the number of considered assignments is very small.

In [21]:
solver = RecursiveBacktrackingWithAC3(australia)
assignment = solver.solve()
print("Assignment", assignment)
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))
print("# considered assignments", solver.counter)

Assignment {'SA': 'B', 'NT': 'G', 'WA': 'R', 'Q': 'R', 'NSW': 'G', 'V': 'R', 'T': 'B'}
Is consistent? True
Is complete? True
# considered assignments 7


In [26]:
#(⬇️)for original sudoku ac3 doesn't give any improvement in number of steps because both ac3 and heauristics solve it in least possible number of steps
#as we discussed it with Dominik Ludwiczak 
solver = RecursiveBacktrackingWithAC3(sudoku)
assignment = solver.solve()
assignment_visualisation(assignment)
print("Assignment", assignment)
print("Is consistent?", sudoku.is_consistent(assignment))
print("Is complete?", sudoku.is_complete(assignment))
print("# considered assignments", solver.counter)

4 8 3 | 9 2 1 | 6 5 7 
9 6 7 | 3 4 5 | 8 2 1 
2 5 1 | 8 7 6 | 4 9 3 
------+-------+------
5 4 8 | 1 3 2 | 9 7 6 
7 2 9 | 5 6 4 | 1 3 8 
1 3 6 | 7 9 8 | 2 4 5 
------+-------+------
3 7 2 | 6 8 9 | 5 1 4 
8 1 4 | 2 5 3 | 7 6 9 
6 9 5 | 4 1 7 | 3 8 2 

Assignment {(0, 2): '3', (0, 0): '4', (0, 1): '8', (0, 3): '9', (0, 4): '2', (0, 5): '1', (0, 6): '6', (0, 7): '5', (0, 8): '7', (1, 0): '9', (1, 1): '6', (1, 2): '7', (1, 3): '3', (1, 4): '4', (1, 5): '5', (1, 6): '8', (1, 7): '2', (1, 8): '1', (2, 0): '2', (2, 1): '5', (2, 2): '1', (2, 3): '8', (2, 4): '7', (2, 5): '6', (2, 6): '4', (2, 7): '9', (2, 8): '3', (3, 0): '5', (3, 1): '4', (3, 2): '8', (3, 3): '1', (3, 4): '3', (3, 5): '2', (3, 6): '9', (3, 7): '7', (3, 8): '6', (4, 0): '7', (4, 1): '2', (4, 2): '9', (4, 3): '5', (4, 4): '6', (4, 5): '4', (4, 6): '1', (4, 7): '3', (4, 8): '8', (5, 0): '1', (5, 1): '3', (5, 2): '6', (5, 3): '7', (5, 4): '9', (5, 5): '8', (5, 6): '2', (5, 7): '4', (5, 8): '5', (6, 0): '3', (6, 1): '7', (6, 2): 

------------
The pictures are from "Artificial Intelligence: A Modern Approach" 3rd ed.

# Constraint satisfaction problem